# PL

"""
Necessary folders and files
PRC_inputs:
    PL_inputs/plantsconfig1 and PL exe and SPORES exe
    target_DB_docking.sdf con columna IX y pasada por OBABEL.


PLANTS exclusively uses the MOL2 file format, thus MOL2-files (including bond connectivity)
must be provided for all input files. PLANTS expects correct MOL2-atom- and
bond-types. This is needed for the correct identification of rotatable bonds and charged
functional groups and may influence docking and virtual screening performance. Please
consider SPORES for this step.
"""

In [1]:
import numpy as np
import subprocess
from subprocess import Popen
from time import sleep
import os
import pandas as pd

In [2]:
#INPUTS
alphabet=['a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z']
target='p85'
water = False
icm_command='/home/pps/icm-browser-pro-3.9-3a/icmbrowserpro64'
N_cores = 3
cwd='/home/pps'
PL_dir = cwd + '/Documents/PLANTS/'
target_dir = PL_dir + target + '/'
rec_dir = target_dir + target + '_rec'

#ARCHIVOS
rec_mol2 = rec_dir + '.mol2'
DB_filename = target_dir+target+'_DB_docking.mol2'

In [3]:
# Lee el el archivo mols_ filename y averigua Nmol

f=open(DB_filename,'r') 
lines=f.readlines() 
ix=[i for i in range(len(lines)) if lines[i]=='@<TRIPOS>MOLECULE\n']
ix.insert(0,-1)
Nmols=len(ix)-1
print('The number of molecules are',Nmols)
f.close()

The number of molecules are 58


# Si tiene waters:

In [6]:
rec_dir_pdb = target_dir + target.upper() + "_1q4x_rec.pdb"

In [9]:
proc= Popen([PL_dir + './SPORES_64bit','--mode','splitpdb',rec_dir_pdb],cwd = target_dir)


                                SPORES        

             Structure PrOtonation and REcognition System

                             version 1.28


author: T. ten Brink

scientific contributors: P. Duchstein, J. Gimmler, O. Korb, M. Zerara,
                         T.E. Exner

contact: tim.tenbrink@uni-konstanz.de


Molecule number: 1
0.048
0.42
0.436
0.436
0.452
0.504
1.016
1.052
splitting
2
protein: 0
ligs: 0
water: 0
metal: 0


In [4]:
def write_boxin ():  
    water_orig = target_dir + "water.mol2"
    with open(water_orig, 'r') as archivo_origen:
        contenido = archivo_origen.read()
    for i in range (N_cores):
        with open(target_dir + "docking"+str(i+1)+'/water.mol2', 'w') as archivo_destino:
            archivo_destino.write((contenido))

# Grid_box:

In [6]:
archivo_box = "/home/pps/Documents/Dockings/box.in"

In [27]:
def write_boxin ():  
    archivo_box = "/home/pps/Documents/Dockings/box.in"
    water_origi = target_dir + "water.mol2"
    with open(archivo_box, 'r') as archivo_origen:
        contenido = archivo_origen.readlines()
    contenido[2] = coordinates_box + "\n" #chequear 
    with open(target_dir + '/box.in', 'w') as archivo_destino:
        archivo_destino.writelines((contenido))

In [11]:
def get_coordinates_box():
    with open(archivo_dtb, 'r') as dtb:
        contenido_dtb = dtb.read().splitlines()

    x1, y1, z1, x2, y2, z2 = float(contenido_dtb[17]), float(contenido_dtb[18]), float(contenido_dtb[19]), float(contenido_dtb[20]), float(contenido_dtb[21]), float(contenido_dtb[22])
    
    # Calculando el centro del cubo
    cx = (x1 + x2) / 2
    cy = (y1 + y2) / 2
    cz = (z1 + z2) / 2
    
    xyz = str(cx) + " " + str(cy) + " " + str(cz)
    print(xyz)

    # Imprimir el centro del cubo
    print(f"Centro del cubo:")
    print(f"cx = {cx}")
    print(f"cy = {cy}")
    print(f"cz = {cz}")
    
    return (str(cx) + " " + str(cy) + " " + str(cz))

In [30]:
archivo_dtb = target_dir + target.upper()  + "_1q4x.dtb"
coordinates_box = get_coordinates_box()
write_boxin()
# editar post ADT las lengths de las box
print("falta editar las longitudes de cada uno en el box.in")

14.4597 22.25715 46.6451
Centro del cubo:
cx = 14.4597
cy = 22.25715
cz = 46.6451
falta editar las longitudes de cada uno en el box.in


# Edito config:

In [32]:
def edit_config(config_filename, N_cores, target, target_dir, rec_dir):
    
    fpl = open(config_filename, 'r')
    lines_fpl = fpl.readlines()
    fpl.close()

    box_in = open(target_dir + 'box.in', 'r')
    lines_box_in = box_in.readlines()
    box_in.close()

# averiguo valores
    x_cent = (lines_box_in[2].split()[0])
    y_cent = (lines_box_in[2].split()[1])
    z_cent = (lines_box_in[2].split()[2])
    bs_rec = int((max(np.array(lines_box_in[3].split()[0:3]).astype(int)) / 2)) # calcula la mitad del valor máximo entre los tres números

# cambio valores
    lines_fpl[6] = 'protein_file ' + target_dir + target + '_rec.mol2 \n'
    lines_fpl[18] = 'bindingsite_center ' + x_cent + ' ' + y_cent + ' ' + z_cent + ' \n'
    lines_fpl[19] = 'bindingsite_radius ' + str(bs_rec) + ' \n'
    
    if os.path.exists(target_dir+"water.mol2"):
        water = True
        print("It has waters.")
        lines_fpl[12] = "#" + lines_fpl[12]
        fwater = open(target_dir+"water.mol2", 'r')
        lines_fwater = fwater.readlines()
        fwater.close()
        lines_fpl[27] = "water_molecule " + lines_fwater[8].split()[2] + " " + lines_fwater[8].split()[3] + " " + lines_fwater[8].split()[4] + " 0 0"+" \n"
        for i in range (28,32):
            lines_fpl[i] = lines_fpl[i].split("#")[1]

    num = np.arange(N_cores + 1)[1::] 
    print(num)
    for i in num:
        lines_fpl[7] = 'ligand_file ' + target + '_DB_' + str(i) + '.mol2 \n'
        lines_fpl[10] = 'output_dir results' + str(i) + ' \n'

        fpl = open(target_dir + 'docking'+ str(i)+'/plantsconfig' + str(i), 'w')
        for line in lines_fpl:
            fpl.write(line)
        fpl.close()

    return

In [34]:
edit_config(PL_dir + 'plantsconfig', N_cores, target, target_dir, rec_dir)
#chekear que se crearon y no estan vacios

[1 2 3 4 5 6 7 8]


# Spores DB y docking:

In [35]:
# corre SPORES y convierte file de alphabet a numeros
for i in range(N_cores):
    DB_dir_docking = target_dir + "docking" + str (i+1) + "/" + target + '_DB_'
    proc= Popen([PL_dir + './SPORES_64bit','--mode','readbonds',DB_dir_docking + alphabet[i] + ".mol2",DB_dir_docking+ str(i+1)+".mol2"])
    sleep(5)


                                SPORES        

             Structure PrOtonation and REcognition System

                             version 1.28


author: T. ten Brink

scientific contributors: P. Duchstein, J. Gimmler, O. Korb, M. Zerara,
                         T.E. Exner

contact: tim.tenbrink@uni-konstanz.de


Molecule number: 1
Molecule number: 2
Molecule number: 3
Molecule number: 4
Molecule number: 5
Molecule number: 6
Molecule number: 7
Molecule number: 8
Molecule number: 9
Molecule number: 10
Molecule number: 11
Molecule number: 12
Molecule number: 13
Molecule number: 14
Molecule number: 15
Molecule number: 16
Molecule number: 17
Molecule number: 18
Molecule number: 19
Molecule number: 20
Molecule number: 21
Molecule number: 22
Molecule number: 23
Molecule number: 24
Molecule number: 25
Molecule number: 26
Molecule number: 27
Molecule number: 28
Molecule number: 29
Molecule number: 30
Molecule number: 31
Molecule number: 32
Molecule number: 33
Molecule number: 34
Molecul

Molecule number: 407
Molecule number: 408
Molecule number: 409
Molecule number: 410
Molecule number: 411
Molecule number: 412
Molecule number: 413
Molecule number: 414
Molecule number: 415
Molecule number: 416
Molecule number: 417
Molecule number: 418
Molecule number: 419
Molecule number: 420
Molecule number: 421
Molecule number: 422
Molecule number: 423
Molecule number: 424
Molecule number: 425
Molecule number: 426
Molecule number: 427
Molecule number: 428
Molecule number: 429
Molecule number: 430
Molecule number: 431
Molecule number: 432
Molecule number: 433
Molecule number: 434
Molecule number: 435
Molecule number: 436
Molecule number: 437
Molecule number: 438
Molecule number: 439
Molecule number: 440
Molecule number: 441
Molecule number: 442
Molecule number: 443
Molecule number: 444
Molecule number: 445
Molecule number: 446
Molecule number: 447
Molecule number: 448
Molecule number: 449
Molecule number: 450
Molecule number: 451
Molecule number: 452
Molecule number: 453
Molecule numb

Molecule number: 806
Molecule number: 807
Molecule number: 808
Molecule number: 809
Molecule number: 810
Molecule number: 811
Molecule number: 812
Molecule number: 813
Molecule number: 814
Molecule number: 815
Molecule number: 816
Molecule number: 817
Molecule number: 818
Molecule number: 819
Molecule number: 820
Molecule number: 821
Molecule number: 822
Molecule number: 823
Molecule number: 824
Molecule number: 825
Molecule number: 826
Molecule number: 827
Molecule number: 828
Molecule number: 829
Molecule number: 830
Molecule number: 831
Molecule number: 832
Molecule number: 833
Molecule number: 834
Molecule number: 835
Molecule number: 836
Molecule number: 837
Molecule number: 838
Molecule number: 839
Molecule number: 840
Molecule number: 841
Molecule number: 842
Molecule number: 843
Molecule number: 844
Molecule number: 845
Molecule number: 846
Molecule number: 847
Molecule number: 848
Molecule number: 849
Molecule number: 850
Molecule number: 851
Molecule number: 852
Molecule numb

Molecule number: 240
Molecule number: 241
Molecule number: 242
Molecule number: 243
Molecule number: 244
Molecule number: 245
Molecule number: 246
Molecule number: 247
Molecule number: 248
Molecule number: 249
Molecule number: 250
Molecule number: 251
Molecule number: 252
Molecule number: 253
Molecule number: 254
Molecule number: 255
Molecule number: 256
Molecule number: 257
Molecule number: 258
Molecule number: 259
Molecule number: 260
Molecule number: 261
Molecule number: 262
Molecule number: 263
Molecule number: 264
Molecule number: 265
Molecule number: 266
Molecule number: 267
Molecule number: 268
Molecule number: 269
Molecule number: 270
Molecule number: 271
Molecule number: 272
Molecule number: 273
Molecule number: 274
Molecule number: 275
Molecule number: 276
Molecule number: 277
Molecule number: 278
Molecule number: 279
Molecule number: 280
Molecule number: 281
Molecule number: 282
Molecule number: 283
Molecule number: 284
Molecule number: 285
Molecule number: 286
Molecule numb

Molecule number: 642
Molecule number: 643
Molecule number: 644
Molecule number: 645
Molecule number: 646
Molecule number: 647
Molecule number: 648
Molecule number: 649
Molecule number: 650
Molecule number: 651
Molecule number: 652
Molecule number: 653
Molecule number: 654
Molecule number: 655
Molecule number: 656
Molecule number: 657
Molecule number: 658
Molecule number: 659
Molecule number: 660
Molecule number: 661
Molecule number: 662
Molecule number: 663
Molecule number: 664
Molecule number: 665
Molecule number: 666
Molecule number: 667
Molecule number: 668
Molecule number: 669
Molecule number: 670
Molecule number: 671
Molecule number: 672
Molecule number: 673
Molecule number: 674
Molecule number: 675
Molecule number: 676
Molecule number: 677
Molecule number: 678
Molecule number: 679
Molecule number: 680
Molecule number: 681
Molecule number: 682
Molecule number: 683
Molecule number: 684
Molecule number: 685
Molecule number: 686
Molecule number: 687
Molecule number: 688
Molecule numb

...

Molecule number: 899
Molecule number: 900
Molecule number: 901
Molecule number: 902
Molecule number: 903
Molecule number: 904
Molecule number: 905
Molecule number: 906
Molecule number: 907
Molecule number: 908
Molecule number: 909
Molecule number: 910
Molecule number: 911
Molecule number: 912
Molecule number: 913
Molecule number: 914
Molecule number: 915
Molecule number: 916
Molecule number: 917
Molecule number: 918
Molecule number: 919
Molecule number: 920
Molecule number: 921
Molecule number: 922
Molecule number: 923
Molecule number: 924
Molecule number: 925
Molecule number: 926
Molecule number: 927
Molecule number: 928
Molecule number: 929
Molecule number: 930
Molecule number: 931
Molecule number: 932
Molecule number: 933
Molecule number: 934
Molecule number: 935
Molecule number: 936
Molecule number: 937
Molecule number: 938
Molecule number: 939
Molecule number: 940
Molecule number: 941
Molecule number: 942
Molecule number: 943
Molecule number: 944
Molecule number: 945
Molecule numb

Molecule number: 325
Molecule number: 326
Molecule number: 327
Molecule number: 328
Molecule number: 329
Molecule number: 330
Molecule number: 331
Molecule number: 332
Molecule number: 333
Molecule number: 334
Molecule number: 335
Molecule number: 336
Molecule number: 337
Molecule number: 338
Molecule number: 339
Molecule number: 340
Molecule number: 341
Molecule number: 342
Molecule number: 343
Molecule number: 344
Molecule number: 345
Molecule number: 346
Molecule number: 347
Molecule number: 348
Molecule number: 349
Molecule number: 350
Molecule number: 351
Molecule number: 352
Molecule number: 353
Molecule number: 354
Molecule number: 355
Molecule number: 356
Molecule number: 357
Molecule number: 358
Molecule number: 359
Molecule number: 360
Molecule number: 361
Molecule number: 362
Molecule number: 363
Molecule number: 364
Molecule number: 365
Molecule number: 366
Molecule number: 367
Molecule number: 368
Molecule number: 369
Molecule number: 370
Molecule number: 371
Molecule numb

Molecule number: 755
Molecule number: 756
Molecule number: 757
Molecule number: 758
Molecule number: 759
Molecule number: 760
Molecule number: 761
Molecule number: 762
Molecule number: 763
Molecule number: 764
Molecule number: 765
Molecule number: 766
Molecule number: 767
Molecule number: 768
Molecule number: 769
Molecule number: 770
Molecule number: 771
Molecule number: 772
Molecule number: 773
Molecule number: 774
Molecule number: 775
Molecule number: 776
Molecule number: 777
Molecule number: 778
Molecule number: 779
Molecule number: 780
Molecule number: 781
Molecule number: 782
Molecule number: 783
Molecule number: 784
Molecule number: 785
Molecule number: 786
Molecule number: 787
Molecule number: 788
Molecule number: 789
Molecule number: 790
Molecule number: 791
Molecule number: 792
Molecule number: 793
Molecule number: 794
Molecule number: 795
Molecule number: 796
Molecule number: 797
Molecule number: 798
Molecule number: 799
Molecule number: 800
Molecule number: 801
Molecule numb

Molecule number: 245
Molecule number: 246
Molecule number: 247
Molecule number: 248
Molecule number: 249
Molecule number: 250
Molecule number: 251
Molecule number: 252
Molecule number: 253
Molecule number: 254
Molecule number: 255
Molecule number: 256
Molecule number: 257
Molecule number: 258
Molecule number: 259
Molecule number: 260
Molecule number: 261
Molecule number: 262
Molecule number: 263
Molecule number: 264
Molecule number: 265
Molecule number: 266
Molecule number: 267
Molecule number: 268
Molecule number: 269
Molecule number: 270
Molecule number: 271
Molecule number: 272
Molecule number: 273
Molecule number: 274
Molecule number: 275
Molecule number: 276
Molecule number: 277
Molecule number: 278
Molecule number: 279
Molecule number: 280
Molecule number: 281
Molecule number: 282
Molecule number: 283
Molecule number: 284
Molecule number: 285
Molecule number: 286
Molecule number: 287
Molecule number: 288
Molecule number: 289
Molecule number: 290
Molecule number: 291
Molecule numb

Molecule number: 686
Molecule number: 687
Molecule number: 688
Molecule number: 689
Molecule number: 690
Molecule number: 691
Molecule number: 692
Molecule number: 693
Molecule number: 694
Molecule number: 695
Molecule number: 696
Molecule number: 697
Molecule number: 698
Molecule number: 699
Molecule number: 700
Molecule number: 701
Molecule number: 702
Molecule number: 703
Molecule number: 704
Molecule number: 705
Molecule number: 706
Molecule number: 707
Molecule number: 708
Molecule number: 709
Molecule number: 710
Molecule number: 711
Molecule number: 712
Molecule number: 713
Molecule number: 714
Molecule number: 715
Molecule number: 716
Molecule number: 717
Molecule number: 718
Molecule number: 719
Molecule number: 720
Molecule number: 721
Molecule number: 722
Molecule number: 723
Molecule number: 724
Molecule number: 725
Molecule number: 726
Molecule number: 727
Molecule number: 728
Molecule number: 729
Molecule number: 730
Molecule number: 731
Molecule number: 732
Molecule numb

In [5]:
num=np.arange(N_cores+1)[1::] #crea un arreglo con n enteros del 1 a Ncores, sin inculir el 0
for i in num:
    subprocess.Popen([PL_dir + './PLANTS1.2_64bit','--mode','screen','plantsconfig'+str(i),">", "/" + target +'_'+str(i)+'.log','&'], cwd = target_dir + "docking" + str(i))






                  PLANTS
        Protein-Ligand ANT System
               version 1.2


                  PLANTS
        Protein-Ligand ANT System
               version 1.2
author: Oliver Korb



scientific contributors: T.E. Exner, T. Stuetzle
author: Oliver Korb


contact: Oliver.Korb@uni-konstanz.de
scientific contributors: T.E. Exner, T. Stuetzle



contact: Oliver.Korb@uni-konstanz.de
run PLANTS: PLANTS --mode screen yourconfigfile



run PLANTS: PLANTS --mode screen yourconfigfile


                  PLANTS
        Protein-Ligand ANT System
               version 1.2


author: Oliver Korb

scientific contributors: T.E. Exner, T. Stuetzle

contact: Oliver.Korb@uni-konstanz.de


run PLANTS: PLANTS --mode screen yourconfigfile



PLANTS error: PLANTS error: Cannot create output directory results2/

Cannot create output directory results1/

PLANTS error: Cannot create output directory results3/



# # Process results
Run only when PL finished

In [6]:
"""
Concatenamos los archivos de todos los folder separados en los N_cores
Creamos archivos:
    * bestranked_ligands_target.csv
    * out_plants_target.mol2
"""

def concatenate():
    
    destination_file = target_dir + "bestranked_ligands_"+target+".csv"
    open(destination_file, 'w').close()

    # Run the command N_cores times
    for i in range(1, N_cores + 1):
        source_file = target_dir + "docking" + str (i) +  "/results" + str(i) + "/ranking/bestranking_ordered.csv"
        
        with open(source_file, 'r') as file_to_read:
            lines_1 = file_to_read.read()

        # Append the contents to the destination CSV file
        with open(destination_file, 'a') as dest_csv:
            dest_csv.write((lines_1))
        
    #checks if file exists
    print(os.path.exists(f'{target_dir}/bestranked_ligands_{target}.csv'))

    #now the .mol2 files
    processes = []
    destination_file = target_dir + "out_plants_"+target+".mol2"
    open(destination_file, 'w').close()

    for i in range(1, N_cores + 1):
        source_file = target_dir + "docking" + str (i) + "/results" + str(i) + "/ranking/ranked_ligands.mol2"
        
        with open(source_file, 'r') as source_csv:
            lines_2 = source_csv.read()

        # Append the contents to the destination CSV file
        with open(destination_file, 'a') as dest_csv:
            dest_csv.write((lines_2))

In [7]:
def concatenate_w_water():
    
    destination_file = target_dir + "bestranked_ligands_"+target+".csv"
    open(destination_file, 'w').close()

    # Run the command N_cores times
    for i in range(1, N_cores + 1):
        source_file = target_dir + "docking" + str (i) +  "/results" + str(i) + "/bestranking.csv"
        
        with open(source_file, 'r') as file_to_read:
            lines_1 = file_to_read.read()

        # Append the contents to the destination CSV file
        with open(destination_file, 'a') as dest_csv:
            dest_csv.write((lines_1))
        
    #checks if file exists
    print(os.path.exists(f'{target_dir}/bestranked_ligands_{target}.csv'))

    
    print("conda activate prc")
    for i in range (N_cores):
        filter_name = str("title='*_conf_01'")
        #print("obabel -imol2 " + target_dir + "docking"+str(i+1)+"/results"+str(i+1)+"/docked_ligands.mol2 -osdf -O "+target_dir+"docking"+str(i+1)+"/results"+str(i+1)+"/docked_ligands.sdf")
        print("obabel "+target_dir+"docking"+str(i+1)+"/results"+str(i+1)+"/docked_ligands.mol2 -omol2 -O "+target_dir+"docking"+str(i+1)+"/bestranked_ligands.mol2 --filter " +'"'+filter_name+'"')

In [9]:
water = True
if water:
    concatenate_w_water()
else:
    concatenate()

True
conda activate prc
obabel /home/pps/Documents/PLANTS/p85/docking1/results1/docked_ligands.mol2 -omol2 -O /home/pps/Documents/PLANTS/p85/docking1/bestranked_ligands.mol2 --filter "title='*_conf_01'"
obabel /home/pps/Documents/PLANTS/p85/docking2/results2/docked_ligands.mol2 -omol2 -O /home/pps/Documents/PLANTS/p85/docking2/bestranked_ligands.mol2 --filter "title='*_conf_01'"
obabel /home/pps/Documents/PLANTS/p85/docking3/results3/docked_ligands.mol2 -omol2 -O /home/pps/Documents/PLANTS/p85/docking3/bestranked_ligands.mol2 --filter "title='*_conf_01'"


In [10]:
#uno archivos de obabel
if water:
    
    destination_file = target_dir + "out_plants_"+target+".mol2"
    open(destination_file, 'w').close()

    for i in range(1, N_cores + 1):
        source_file = target_dir + "docking" + str (i) +  "/bestranked_ligands.mol2"
        
        with open(source_file, 'r') as source_csv:
            lines_2 = source_csv.read()

        # Append the contents to the destination CSV file
        with open(destination_file, 'a') as dest_csv:
            dest_csv.write((lines_2))
            
    print(os.path.exists(destination_file))

True


In [11]:
# trsform mol2 to sdf
destination_file = target_dir + "out_plants_"+target+".mol2"
out_file = target_dir + "out_"+target+"_pl.sdf"
print("conda activate prc")
print('obabel -imol2 '+ destination_file + ' -osdf -O '+ out_file)

conda activate prc
obabel -imol2 /home/pps/Documents/PLANTS/p85/out_plants_p85.mol2 -osdf -O /home/pps/Documents/PLANTS/p85/out_p85_pl.sdf


In [12]:
#ruta = 'Documents/PLANTS/esr1/results1/ranking/'
df=pd.read_csv(target_dir + 'bestranked_ligands_' + target + '.csv', sep=',', skiprows=1, header=None)
df = df [[0,1]] # me quedo con el nombre de la molecula y con el TOTAL SCORE
df

,0,1
0,AD-M1-133_1 1_entry_00001_conf_01,-64.6496
1,AD-M1-133_2 2_entry_00002_conf_01,-63.7029
2,AD-M1-149_1 3_entry_00003_conf_01,-63.7155
3,AD-M1-149_2 4_entry_00004_conf_01,-62.5831
4,AD-M1-149M_1 5_entry_00005_conf_01,-71.1574
5,AD-M1-149M_2 6_entry_00006_conf_01,-69.2826
6,BPH10 7_entry_00007_conf_01,-48.3844
7,BPH11 8_entry_00008_conf_01,-48.0816
8,BPH12 9_entry_00009_conf_01,-48.1132
9,BPH1 10_entry_00010_conf_01,-48.4876


In [13]:
dr=df.loc[df[0]=='LIGAND_ENTRY']
dr

,0,1
19,LIGAND_ENTRY,TOTAL_SCORE
39,LIGAND_ENTRY,TOTAL_SCORE


In [14]:
df=df.drop(list(dr.index))
df[0]=[int(df.iloc[i][0].split(' ')[1].split('_')[0]) for i in range(len(df))]
df

,0,1
0,1,-64.6496
1,2,-63.7029
2,3,-63.7155
3,4,-62.5831
4,5,-71.1574
5,6,-69.2826
6,7,-48.3844
7,8,-48.0816
8,9,-48.1132
9,10,-48.4876


In [15]:
dict_ix_scores={int(df.iloc[i][0]):df.iloc[i][1] for i in range(len(df))}
dict_ix_scores

{1: '-64.6496',
 2: '-63.7029',
 3: '-63.7155',
 4: '-62.5831',
 5: '-71.1574',
 6: '-69.2826',
 7: '-48.3844',
 8: '-48.0816',
 9: '-48.1132',
 10: '-48.4876',
 11: '-51.1332',
 12: '-46.8473',
 13: '-47.6364',
 14: '-52.284',
 15: '-49.4473',
 16: '-51.1415',
 17: '-52.7802',
 18: '-48.4686',
 19: '-59.1172',
 20: '-61.51',
 21: '-58.8159',
 22: '-58.7208',
 23: '-72.3667',
 24: '-72.0293',
 25: '-67.8015',
 26: '-69.6603',
 27: '-63.8563',
 28: '-61.9996',
 29: '-61.7643',
 30: '-56.2505',
 31: '-57.5937',
 32: '-52.2239',
 33: '-55.8063',
 34: '-57.4108',
 35: '-69.5282',
 36: '-69.0468',
 37: '-69.2202',
 38: '-67.2283',
 39: '-67.0272',
 40: '-69.6306',
 41: '-69.7028',
 42: '-71.6903',
 43: '-68.0599',
 44: '-68.8334',
 45: '-72.4154',
 46: '-64.5916',
 47: '-66.0716',
 48: '-64.8143',
 49: '-63.3844',
 50: '-68.8036',
 51: '-67.6777',
 52: '-70.8709',
 53: '-71.0509',
 54: '-68.741',
 55: '-69.652',
 56: '-75.5048',
 57: '-66.5944',
 58: '-66.0002'}

In [16]:
f=open(target_dir + 'out_' + target + '_pl.sdf', 'r') 
lines=f.readlines()
ix=[i for i in range(len(lines)) if lines[i]=='$$$$\n']
ix.insert(0,-1) 
print(len(ix)) 

Nmols=len(ix)-1
print('Number of molecules:',Nmols)
f.close()

53
Number of molecules: 52


In [17]:
i = 0
mol=lines[ix[i]+1:ix[i+1]-1] 
mol

['AD-M1-133_1 1_entry_00001_conf_01\n',
 ' OpenBabel12232416303D\n',
 '\n',
 ' 41 44  0  0  1  0  0  0  0  0999 V2000\n',
 '   36.1215   31.3382   25.4688 O   0  0  0  0  0  0  0  0  0  0  0  0\n',
 '   36.2701   31.1023   24.2714 C   0  0  0  0  0  0  0  0  0  0  0  0\n',
 '   35.3987   30.3165   23.5783 N   0  0  0  0  0  0  0  0  0  0  0  0\n',
 '   35.4559   29.9775   22.2469 C   0  0  0  0  0  0  0  0  0  0  0  0\n',
 '   34.3213   29.0835   21.7283 C   0  0  0  0  0  0  0  0  0  0  0  0\n',
 '   33.4242   29.8362   20.7060 N   0  3  1  0  0  0  0  0  0  0  0  0\n',
 '   34.2746   30.4462   19.5645 C   0  0  0  0  0  0  0  0  0  0  0  0\n',
 '   33.3659   31.2130   18.5661 C   0  0  2  0  0  0  0  0  0  0  0  0\n',
 '   32.2956   30.3119   18.1176 N   0  0  0  0  0  0  0  0  0  0  0  0\n',
 '   32.4660   29.8086   16.8726 C   0  0  0  0  0  0  0  0  0  0  0  0\n',
 '   33.6953   30.5177   16.2990 C   0  0  0  0  0  0  0  0  0  0  0  0\n',
 '   34.0230   31.6427   17.2670 C   0  0 

In [18]:
new_file=[]
for i in range(Nmols):
    mol=lines[ix[i]+1:ix[i+1]-1] 
    IX=int(mol[0].split(' ')[1].split('_')[0]) 
    mol.append('M  END\n') 
    mol.append('>  <IX> \n')
    mol.append(str(IX)+' \n')
    mol.append('>  <Type> \n')    
    # OPCION A
    if IX < 67:
        mol.append('L \n')
    else:
        mol.append('D \n')
    
    mol.append('>  <Score_PL> \n')
    mol.append(str(dict_ix_scores[IX])+' \n')
    mol.append(' \n')
    mol.append('$$$$\n') 
    for line in mol: 
        new_file.append(line)


In [19]:
f=open(target_dir + 'out_' + target + '_pl.sdf', 'w')
for line in new_file:
    f.write(line)
f.close()

Now the use of ICM is required to sort the molecules by molecule number
this is a script for then be run inm ICM
esto lo podemos correr directo desde la consola de ICM gui, y ahi vamos a visualizar la tabal con las columnas
Para saber como se llama real la columna, nos paramos arriba del nombre y hacemos click derecho

In [20]:
f=open('sort.icm','w')
f.write('#!/usr/bin/icm \n')
f.write('call _startup \n')
f.write('\n')
f.write('maxFileSizeMb=2500 \n')
f.write('read table mol \"' + target_dir + 'out_' + target + '_pl.sdf\" \n')
f.write('sort out_' + target + '_pl.IX \n')
f.write('write table mol compress out_' + target +'_pl' +' \"' +  target_dir+ 'out_' + target + '_pl.sdf\" \n')
#f.write('quit \n')
f.close()

In [21]:
subprocess.Popen([icm_command,'-g','sort.icm'])

<Popen: returncode: None args: ['/home/pps/icm-browser-pro-3.9-3a/icmbrowser...>

 Info>  127 colors and 18 graphic modes read from /home/pps/icm-browser-pro-3.9-3a/icm.clr
Startup> Loading config file /home/pps/.browser/config/icm.cfg ..
